# Test on Essentiality of eciML1515
Try to remove some proteins (upper bound to 0) and see if the output of the model.  

First, download the model from the repo.

In [1]:
! [ ! -f "eciML1515.xml" ] && curl -O -L "https://raw.githubusercontent.com/BenjaSanchez/notebooks/master/eciML1515.xml"

In [2]:
import cobra
import sys

model = cobra.io.read_sbml_model("eciML1515.xml")
model.solver.timeout = 5
model.optimize()

,fluxes,reduced_costs
EX_acgam_e,0.000000,-0.282126
EX_cellb_e,0.000000,-0.361121
EX_chol_e,0.000000,-0.022570
EX_pi_e,0.000000,0.000000
EX_h_e,8.058201,0.000000
...,...,...
prot_Q59385_exchange,0.000000,-0.000000
prot_Q6BEX0_exchange,0.000000,-0.000000
prot_Q6BF16_exchange,0.000000,-0.000000
prot_Q6BF17_exchange,0.000000,-0.000000


In [3]:
prot_exchanges = [reac for reac in model.reactions if reac.id.startswith("prot_") and not reac.id.endswith("REV")]

In [4]:
class ProBar:
    """
    Just a simple progress bar object to output on screen
    """

    def __init__(self, n, blength=20):
        self.n = n
        self.blength = 20
        self.progress(0)

    def progress(self, pro):
        pro = pro / self.n
        block = int(round(self.blength * pro))
        bar = (
            "\rRunning... ["
            + "#" * block
            + "-" * (self.blength - block)
            + "] "
            + "{:.2f}".format(pro * 100)
            + "%"
        )
        if pro == 1:
            bar += " DONE!\n"
        # This works on a shell. On an IDE is quite cumbersome.
        sys.stdout.write(bar)
        sys.stdout.flush()

In [9]:
import math

bar = ProBar(len(prot_exchanges))

for i, protr in enumerate(prot_exchanges):
    bar.progress(i)
    protr.bounds = 0,0
    sol = model.slim_optimize()
    if math.isnan(sol):
        essential_protein_ex = protr
        break
        
essential_protein_ex

Running... [--------------------] 0.00%

Reaction identifier,prot_O32583_exchange
Name,prot_O32583_exchange
Memory address,0x07f983a7f7f28
Stoichiometry,--> prot_O32583__91__c__93__ --> prot_O32583 [cytosol]
GPR,b4407
Lower bound,0
Upper bound,0
